# Agents

You can pass a Runnable into an agent.

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain_community.chat_models import ChatAnthropic

In [3]:
model = ChatAnthropic(model="claude-2")

In [4]:
@tool
def search(query: str) -> str:
    """Search things about current events."""
    return "32 degrees"

In [5]:
tool_list = [search]

In [6]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo")

In [7]:
# Logic for going from intermediate steps to a string to pass into model
# This is pretty tied to the prompt
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log


# Logic for converting tools to string to go in prompt
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

Building an agent from a runnable usually involves a few things:

1. Data processing for the intermediate steps. These need to represented in a way that the language model can recognize them. This should be pretty tightly coupled to the instructions in the prompt

2. The prompt itself

3. The model, complete with stop tokens if needed

4. The output parser - should be in sync with how the prompt specifies things to be formatted.

In [12]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [14]:
agent_executor.invoke({"input": "whats the weather in New york?"})



> Entering new AgentExecutor chain...
 <tool>search</tool><tool_input>weather in New York32 degrees <tool>search</tool>
<tool_input>weather in New York32 degrees <final_answer>The weather in New York is 32 degrees

> Finished chain.


{'input': 'whats the weather in New york?',
 'output': 'The weather in New York is 32 degrees'}